In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("preprocessed_1.csv")

In [3]:
df.columns

Index(['raw_id', 'clinic_application_date', 'intake_date', 'signing_date',
       'suiteable_for_pp', 'First name', 'Clinic', 'sheet', 'clean_id',
       'group', 'therapy_start_date', 'therapy_end_date'],
      dtype='object')

In [4]:
priority = {
        'סיימו טיפול': 0,
        'CAU': 1,
        'IPC-SSC': 2,
        'משתתפים פעילים': 3,
        'פספוסי גיוסים': 4,
        'אין שת"פ טיפולי': 5,
        'אי התאמה למחקר': 6,
        'אי הסכמה למחקר': 7,
        'נשירה מחקרית': 8,
        'נשירה קלינית- לאחר ת. טיפול': 9,
        'עלייה לרמה 2': 10
}

In [5]:
formats = [
    "%Y-%m-%d",   # 2025-09-28
    "%d/%m/%Y",   # 28/09/2025
    "%m/%d/%Y",   # 09/28/2025
    "%d-%b-%Y",   # 28-Sep-2025
    "%d.%m.%Y",   # 28.09.2025
    '%Y-%m-%d %H:%M:%S',
    "%d.%m.%y",
    "YYYY-MM-DD HH:MM:SS",
    "%Y-%m-%d %H:%M:%S"



]

date_columns = [i for i in df.columns if 'date' in i]


In [6]:
def parse_date(x):
    for fmt in formats:
        try:
            if type(x) == str:
                x = x.strip()
            return pd.to_datetime(x, format=fmt)
        except (ValueError, TypeError):
            continue
    return pd.NaT

for date_col in date_columns:
    df[date_col] = df[date_col].apply(parse_date)

#### Filling missing data by prioraty 

In [7]:
# map priority and sort so preferred sources come first for each participant
df['prio'] = df['sheet'].map(priority)
df_sorted = df.sort_values(['clean_id', 'prio'])

# group by participant and take the first non-null value per column
# groupby.first() skips NA and returns first non-NA per column
result = df_sorted.groupby('clean_id', as_index=False).first()

# drop helper column if you like
result = result.drop(columns=['prio'])

In [8]:
dummies = pd.get_dummies(df['sheet'])
dummies['clean_id'] = df['clean_id']
one_hot = dummies.groupby('clean_id', as_index=True).max().reset_index()

enriched = df.merge(one_hot, on="clean_id", how="left")
enriched[dummies.columns] = enriched[dummies.columns].fillna(0)


In [9]:

enriched["first_contact_date"] = (
    enriched["intake_date"]
    .fillna(enriched["clinic_application_date"])
    .fillna(enriched["signing_date"])
)
enriched["therapy_starting_date"] = enriched["therapy_start_date"]




In [11]:
enriched.to_csv(r"preprocessed_2.csv", index=0)

In [12]:
final2['group'].unique()

array(['CAU', 0, 'Stepped Care', 'טיפול בקבוצת הורים '], dtype=object)

In [13]:
final2.columns

Index(['clean_id', 'raw_id', 'clinic_application_date', 'intake_date',
       'signing_date', 'suiteable_for_pp', 'First name', 'sheet', 'group',
       'therapy_start_date', 'therapy_end_date', 'אי הסכמה למחקר',
       'אי התאמה למחקר', 'אין שת"פ טיפולי', 'משתתפים פעילים', 'נשירה מחקרית',
       'נשירה קלינית- לאחר ת. טיפול', 'סיימו טיפול', 'עלייה לרמה 2',
       'פספוסי גיוסים', 'first_contact_date', 'therapy_starting_date'],
      dtype='object')